In [1]:
dir = dirname(pwd()) * "/PROJECT"

using Pkg
Pkg.activate(dir)

using Distributed

  Activating project at `C:\Users\conno\Documents\CAMBRIDGE\Part III\Comp Project\JULIA\PROJECT`


In [2]:
global const multiProcess = (nworkers()>1) ? true : false # only use multiprocessing if run with -p, otherwise use -t threads by default

if multiProcess
    print(nworkers())
    
    @everywhere using Pkg
    @everywhere Pkg.activate(dir)
    @everywhere using Distributed, StatsBase, Statistics, Distributions, Roots, PyPlot, LsqFit, Dates # , ProfileVega # , Bootstrap #, ProgressMeter, ProfileVega, JLD
else
    print(Threads.nthreads())
    
    using StatsBase, Statistics, Distributions, Roots, PyPlot, LsqFit, Dates # , ProfileVega # , Bootstrap #, ProgressMeter, ProfileVega, JLD
end

using Colors, PlotUtils, Graphs

6

In [3]:
@everywhere global const sixVertex::Bool = false
@everywhere global const twoFlip::Bool = true
@everywhere global const δE::Int = sixVertex ? 8 : 4

In [11]:
@everywhere include(dir * "/functions/DataStructure.jl")
@everywhere include(dir * "/functions/Plotting.jl")
@everywhere include(dir * "/functions/Statistics.jl")
@everywhere include(dir * "/functions/Simulation.jl")

In [12]:
@everywhere include(dir * "/functions/simulationFunctions/MicroKubo.jl")

LoadError: TaskFailedException

[91m    nested task error: [39mSystemError: opening file "C:\\Users\\conno\\Documents\\CAMBRIDGE\\Part III\\Comp Project\\JULIA\\PROJECT\\functions\\MicroKubo.jl": No such file or directory
    Stacktrace:
      [1] [0m[1msystemerror[22m[0m[1m([22m[90mp[39m::[0mString, [90merrno[39m::[0mInt32; [90mextrainfo[39m::[0mNothing[0m[1m)[22m
    [90m    @ [39m[90mBase[39m [90m.\[39m[90m[4merror.jl:176[24m[39m
      [2] [0m[1m#systemerror#80[22m
    [90m    @ [39m[90m.\[39m[90m[4merror.jl:175[24m[39m[90m [inlined][39m
      [3] [0m[1msystemerror[22m
    [90m    @ [39m[90m.\[39m[90m[4merror.jl:175[24m[39m[90m [inlined][39m
      [4] [0m[1mopen[22m[0m[1m([22m[90mfname[39m::[0mString; [90mlock[39m::[0mBool, [90mread[39m::[0mNothing, [90mwrite[39m::[0mNothing, [90mcreate[39m::[0mNothing, [90mtruncate[39m::[0mNothing, [90mappend[39m::[0mNothing[0m[1m)[22m
    [90m    @ [39m[90mBas

In [6]:
L = [10, 10]
PBC = [true, true]
therm_runtime = 1000
runtime = 1000
num_histories = 10
tau = 2:floor(Int64, 0.75*runtime)
𝒽 = range(0.0, 2.0, length=7)

T = range(0.01, 10.0, length=20);

In [9]:
pairCount = zeros(Int64, length(T), length(𝒽), num_histories)

for i in 1:num_histories
    for j in eachindex(T)
        for k in eachindex(𝒽)
            vertices, edges = CubicGrid(L, PBC);
            MicroKuboSetup(vertices, edges, therm_runtime, T[j], 𝒽, false)

            # Find all particles
            prtclIndices = []
            for (i,v) in enumerate(vertices)
                Ai = A(edges, v)
                if Ai < 0
                    push!(prtclIndices, i)
                end
            end

            # Find pairs

            pairCount = 0
            for i in prtclIndices
                for j in prtclIndices
                    if i != j
                        Δ = vertices[i].x - vertices[j].x
                        D = 0  
                        # correct for PBCs
                        for d in 1:length(Δ)
                            Δ[d] /= (abs(Δ[d])==L[d]-1) ? -abs(Δ[d]) : 1
                            D += Δ[d]^2
                        end

                        if D == 1
                            pairCount[k,j,i] += 1
                        end
                    end
                end
            end

            #pairCount[k,j,i] /= 2 # correct for 2x-counting
            pairCount[k,j,i] /= length(prtclIndices) # *fraction* of particles that are in pairs
        end
    end
end

LoadError: UndefVarError: MicroKuboSetup not defined

In [10]:
pairCount = mean(pairCount, dims=3)

for k in eachindex(𝒽)
    plot(T, pairCount[k, :])
end

LoadError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\conno\.julia\packages\PyCall\twYvK\src\pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError('x and y must have same first dimension, but have shapes (20,) and (7,)')
  File "C:\Users\conno\.julia\conda\3\lib\site-packages\matplotlib\pyplot.py", line 2740, in plot
    return gca().plot(
  File "C:\Users\conno\.julia\conda\3\lib\site-packages\matplotlib\axes\_axes.py", line 1662, in plot
    lines = [*self._get_lines(*args, data=data, **kwargs)]
  File "C:\Users\conno\.julia\conda\3\lib\site-packages\matplotlib\axes\_base.py", line 311, in __call__
    yield from self._plot_args(
  File "C:\Users\conno\.julia\conda\3\lib\site-packages\matplotlib\axes\_base.py", line 504, in _plot_args
    raise ValueError(f"x and y must have same first dimension, but "


Obviously the above will fail when there are multiple **overlapping** pairs, but at low temperatures it should give us a decent estimate.

In terms of an analytic expectation, I'm really not sure. Recalling that particles are generated from the GS by creating blue spin-down strings, we should seek to minimise these strings and therefore increase the number of particles which exist in pairs.